# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver (No caso do chrome, é chromedriver. No caso do firefox, é geckodriver)

In [23]:
from selenium import webdriver as wbd
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.chrome.options import Options

#weboptions = Options()
#weboptions.headless = True

#browser = wbd.Chrome(options=weboptions)
browser = wbd.Chrome()

# 1 - Pegar cotação do dólar
# Entrar no google
browser.get('https://www.google.com')
browser.find_element('xpath', 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do dólar')
browser.find_element(
'xpath', 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = browser.find_element(
'xpath', 
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print("Cotação do dólar: R$", cotacao_dolar)

# 2 - Pegar cotação do euro
browser.get('https://www.google.com')
browser.find_element(
'xpath', 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do euro')
browser.find_element(
'xpath', 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = browser.find_element(
'xpath', 
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print("Cotação do euro: R$", cotacao_euro)

# 3 - Pegar cotação do ouro
browser.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = browser.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", '.')
print("Cotação do ouro: R$", cotacao_ouro)

browser.quit()



#https://www.youtube.com/watch?v=g-zVpeJtkFk





Cotação do dólar: R$ 5.1232
Cotação do euro: R$ 5.545556608
Cotação do ouro: R$ 313.22


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [24]:
# 4 - Importar a base de dados e atualizá-la

import pandas as pd

table = pd.read_excel("Produtos.xlsx")
display(table)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [25]:
# 5 - Recalcular os preços

# atualizar a cotação 

table.loc[table.Moeda == 'Dólar', "Cotação"] = float(cotacao_dolar)
table.loc[table.Moeda == 'Euro', "Cotação"] = float(cotacao_euro)
table.loc[table.Moeda == 'Ouro', "Cotação"] = float(cotacao_ouro)# ou table["Moeda"] == 'Dólar'

# preço de compra = cotação * preço original
table['Preço de Compra'] = table['Cotação'] * table['Preço Original']

# preço de venda = preço de compra * margem
table['Preço de Venda'] = table['Preço de Compra'] * table['Margem']

table['Preço de Venda'] = table['Preço de Venda'].map("R${:.2f}".format)
display(table)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.123200,5123.148768,1.40,R$7172.41
1,Carro Renault,4500.00,Euro,5.545557,24955.004736,2.00,R$49910.01
2,Notebook Dell,899.99,Dólar,5.123200,4610.828768,1.70,R$7838.41
3,IPhone,799.00,Dólar,5.123200,4093.436800,1.70,R$6958.84
4,Carro Fiat,3000.00,Euro,5.545557,16636.669824,1.90,R$31609.67
5,Celular Xiaomi,480.48,Dólar,5.123200,2461.595136,2.00,R$4923.19
6,Joia 20g,20.00,Ouro,313.220000,6264.400000,1.15,R$7204.06


### Agora vamos exportar a nova base de preços atualizada

In [26]:
# 6 - Exportar a base atualizada
table.to_excel("ProdutosAtualizados.xlsx", index=False)